In [11]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
from nltk.tokenize import word_tokenize
import statsmodels.api as sm
from statsmodels.formula.api import ols

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

# special matplotlib argument for improved plots
from matplotlib import rcParams
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/raymondeds/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
df_reviews = pd.read_csv('../data/reviews.csv')
df_submissions = pd.read_csv('../data/Submissions.csv')
df_dec = pd.read_csv('../data/decision.csv')
df_keyword = pd.read_csv('../data/submission_keyword.csv')

df_rs = pd.merge(df_reviews[['rating_int','confidence_int','review','forum']], df_submissions[['abstract','id','conf_year']], left_on='forum', right_on='id', how = 'inner')
df_rs['review'] = df_rs['review'].str.lower()
df_rs['abstract'] = df_rs['abstract'].str.lower()
df_rs['review_tokens'] = df_rs['review'].apply(word_tokenize)
df_rs['review_num_tokens'] = df_rs['review_tokens'].apply(len)

df_data = pd.merge(df_rs, df_dec, left_on='forum', right_on='forum', how = 'inner').drop([
                                                         'id_x',
                                                         'id_y',
                                                         'tcdate',
                                                         'tmdate',
                                                         'number',
                                                         'confidence'],axis = 1)

df_data

,rating_int,confidence_int,review,forum,abstract,conf_year,review_tokens,review_num_tokens,replyto,title,decision,comment,conf_name
0,6,4.0,there is a lot of recent work on link-predicti...,tGZu6DlbreV,this paper studies learning logic rules for re...,2021,"[there, is, a, lot, of, recent, work, on, link...",1537,tGZu6DlbreV,Final Decision,Accept (Poster),There is a consensus among the reviewers that ...,ICLR2021
1,7,4.0,"in this paper, the author proposes rnnlogic fo...",tGZu6DlbreV,this paper studies learning logic rules for re...,2021,"[in, this, paper, ,, the, author, proposes, rn...",722,tGZu6DlbreV,Final Decision,Accept (Poster),There is a consensus among the reviewers that ...,ICLR2021
2,6,2.0,this paper focuses on learning logic rules via...,tGZu6DlbreV,this paper studies learning logic rules for re...,2021,"[this, paper, focuses, on, learning, logic, ru...",236,tGZu6DlbreV,Final Decision,Accept (Poster),There is a consensus among the reviewers that ...,ICLR2021
3,8,1.0,"in this work, the authors illustrate an approa...",tGZu6DlbreV,this paper studies learning logic rules for re...,2021,"[in, this, work, ,, the, authors, illustrate, ...",121,tGZu6DlbreV,Final Decision,Accept (Poster),There is a consensus among the reviewers that ...,ICLR2021
4,6,4.0,pros:\n\n- the different attention techniques ...,uKhGRvM8QNH,"knowledge distillation, in which a student mod...",2021,"[pros, :, -, the, different, attention, techni...",338,uKhGRvM8QNH,Final Decision,Accept (Poster),"After the rebuttal stage, all reviewers lean p...",ICLR2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25163,4,4.0,the author proposes the use of low-rank matrix...,rkaRFYcgl,deep learning consists in training neural netw...,2017,"[the, author, proposes, the, use, of, low-rank...",383,rkaRFYcgl,ICLR committee final decision,Reject,The reviewers seem to agree that the framework...,ICLR2017
25164,5,NaN,the paper proposes a low-rank version of pass-...,rkaRFYcgl,deep learning consists in training neural netw...,2017,"[the, paper, proposes, a, low-rank, version, o...",82,rkaRFYcgl,ICLR committee final decision,Reject,The reviewers seem to agree that the framework...,ICLR2017
25165,6,4.0,the authors study the use of low-rank approxim...,rkaRFYcgl,deep learning consists in training neural netw...,2017,"[the, authors, study, the, use, of, low-rank, ...",209,rkaRFYcgl,ICLR committee final decision,Reject,The reviewers seem to agree that the framework...,ICLR2017
25166,6,4.0,​there have been numerous works ​on learning f...,BkUDvt5gg,this paper presents a simple end-to-end model ...,2017,"[​there, have, been, numerous, works, ​on, lea...",326,BkUDvt5gg,ICLR committee final decision,Reject,Without revisions to this paper or a rebuttal ...,ICLR2017


In [9]:
# Step 2: Import the necessary libraries
from transformers import pipeline
from transformers import AutoTokenizer

# Step 3: Load the pre-trained model and tokenizer
sentiment_analysis = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

# Step 4: Define the text for sentiment analysis
text = "I love using Hugging Face's Transformers library! It's amazing."

# Step 5: Perform sentiment analysis
result = sentiment_analysis(text)

# Step 6: Display the result
print(result)

Downloading: 100%|█████████████████████████████| 929/929 [00:00<00:00, 2.76MB/s]
Downloading: 100%|███████████████████████████| 501M/501M [00:11<00:00, 42.0MB/s]
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|███████████████████████████| 899k/899k [00:00<00:00, 3.55MB/s]
Downloading: 100%|████████████████████████

[{'label': 'positive', 'score': 0.9873846769332886}]


In [14]:
sentiment_analysis(df_data.review[0])

RuntimeError: The expanded size of the tensor (1721) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1721].  Tensor sizes: [1, 514]

In [ ]:
def split_text(text, tokenizer, max_length=512):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(tokenizer.encode(chunk + sentence, add_special_tokens=False)) <= max_length:
            chunk += " " + sentence
        else:
            chunks.append(chunk.strip())
            chunk = sentence
    if chunk:
        chunks.append(chunk.strip())
    return chunks

# Step 8: Perform sentiment analysis on each chunk
results = [sentiment_analysis(chunk) for chunk in chunks]

# Step 9: Aggregate the results
positive_score = sum(res[0]['score'] for res in results if res[0]['label'] == 'POSITIVE')
negative_score = sum(res[0]['score'] for res in results if res[0]['label'] == 'NEGATIVE')
